# TREC-6: Simulations

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0
# !pip install ax-platform==0.1.20
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../..")) # comment this if library is pip installed

In [3]:
import io
import re
import pickle

from tqdm.notebook import tqdm

from timeit import default_timer as timer

import random
import numpy as np

import torch
import torch.optim as optim

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorWithPadding

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [4]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
SEED = 42

## Global variables

In [7]:
RESULTS_PATH = '/results/ESN'       # path of your result folder
CACHE_DIR = '/huggingface_datasets' # path of your folder
PARAMS_FILE = 'trec-6_params.pkl'
RESULTS_FILE_1 = 'trec-6_results_esn.pkl'
RESULTS_FILE_2 = 'trec-6_results_cbs.pkl'

## Dataset

In [8]:
# rename correct column as 'labels': depends on the dataset you load

def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_enrich_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes (use 'label-fine' for fine-grained labels)
    dataset = dataset.rename_column('label-coarse', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

full_dataset = load_and_enrich_dataset('trec', split=None, cache_dir=CACHE_DIR)
train_dataset = full_dataset['train'].sort("lengths")
test_dataset = full_dataset['test'].sort("lengths")

dataset_d = {
    'train': train_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [10]:
dataset_d

{'train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 5452
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 500
 })}

## Simulations

- Parameters

In [11]:
# load params

with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'rb') as fh:
    params_l = pickle.load(fh)

In [12]:
# get best params

params_sorted_l = sorted(params_l, key = lambda i: i['accuracy_mean'], reverse=True)
best_params = params_sorted_l[0]['params']

In [13]:
best_params

{'reservoir_dim': 1000,
 'spectral_radius': 1.5,
 'leaking_rate': 0.1,
 'input_scaling': 0.1,
 'bias_scaling': 0.0,
 'sparsity': 0.0,
 'activation_function': 'relu',
 'alpha': 10.0}

In [14]:
# fill up ESN with best parameters

esn_params = {
            'embedding_weights': 'bert-base-uncased', # TEXT.vocab.vectors,
            'distribution' : 'gaussian',              # uniform, gaussian
            'input_dim' : 768,                        # dim of BERT encoding!
            'reservoir_dim' : None,
            'bias_scaling' : None,
            'sparsity' : None,
            'spectral_radius' : None,
            'leaking_rate': None,
            'activation_function' : None,
            'input_scaling' : None,
            'mean' : 0.0,
            'std' : 1.0,
            'merging_strategy' : 'mean',
            'bidirectional' : False,
            'device' : device,
            'mode' : None,
            'seed' : None
             }

# Fill up the reservoir
for k, v in best_params.items():
    if k in esn_params.keys():
        esn_params[k] = v

In [15]:
esn_params

{'embedding_weights': 'bert-base-uncased',
 'distribution': 'gaussian',
 'input_dim': 768,
 'reservoir_dim': 1000,
 'bias_scaling': 0.0,
 'sparsity': 0.0,
 'spectral_radius': 1.5,
 'leaking_rate': 0.1,
 'activation_function': 'relu',
 'input_scaling': 0.1,
 'mean': 0.0,
 'std': 1.0,
 'merging_strategy': 'mean',
 'bidirectional': False,
 'device': device(type='cuda'),
 'mode': None,
 'seed': None}

- Simulations

In [16]:
# simulations (ESN and CBS)

esn_results_d = {}
esn_results_d['params'] = best_params
esn_results_d['reservoir_dim'] = {}

cbs_results_d = {}
cbs_results_d['params'] = best_params
cbs_results_d['reservoir_dim'] = {}

# loop over seeds
for reservoir_dim in tqdm([500, 1000, 3000, 5000]):

    esn_results_d['reservoir_dim'][reservoir_dim] = {}
    esn_results_d['reservoir_dim'][reservoir_dim]['accuracy'] = []
    esn_results_d['reservoir_dim'][reservoir_dim]['time'] = []

    cbs_results_d['reservoir_dim'][reservoir_dim] = {}
    cbs_results_d['reservoir_dim'][reservoir_dim]['accuracy'] = []
    cbs_results_d['reservoir_dim'][reservoir_dim]['time'] = []

    # loop over seeds
    for seed in tqdm([42, 127, 74684, 888, 7716843]):

        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)
        
        # loop over modes
        for mode in tqdm(['esn', 'linear_layer']):

            # ESN parameters
            esn_params['reservoir_dim'] = reservoir_dim
            esn_params['seed'] = seed
            esn_params['mode'] = mode

            # Instantiate the ESN
            ESN = esn.EchoStateNetwork(**esn_params)
            ESN.learning_algo = la.RidgeRegression(alpha=best_params['alpha'])
            ESN = ESN.to(device)

            # Warm up the ESN
            nb_sentences = 10

            for i in range(nb_sentences): 
                sentence = dataset_d["train"].select([i])
                dataloader_tmp = torch.utils.data.DataLoader(sentence, 
                                                             batch_size=1, 
                                                             collate_fn=DataCollatorWithPadding(tokenizer))  

                for sentence in dataloader_tmp:
                    ESN.warm_up(sentence)

            # train on train set
            t0 = timer()
            ESN.fit(dataloader_d["train"])
            t1 = timer()
            time = t1 - t0
            print(f'training time: {time}')

            # accuracy on test set
            test_pred, test_acc = ESN.predict(dataloader_d["test"], verbose=False)
            print(f'accuracy: {test_acc.item()}')

            # save results
            if mode == 'esn':
                esn_results_d['reservoir_dim'][reservoir_dim]['time'].append(time)
                esn_results_d['reservoir_dim'][reservoir_dim]['accuracy'].append(test_acc.item())

            elif mode == 'linear_layer':
                cbs_results_d['reservoir_dim'][reservoir_dim]['time'].append(time)
                cbs_results_d['reservoir_dim'][reservoir_dim]['accuracy'].append(test_acc.item())

            # clean objects
            del ESN
            torch.cuda.empty_cache()


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 4.021726120263338
accuracy: 89.20000457763672


...

  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 8.476667341776192
accuracy: 71.60000610351562


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
training time: 10.213073121383786
accuracy: 93.60000610351562


- Results

In [18]:
for k, v in esn_results_d['reservoir_dim'].items():
    esn_results_d['reservoir_dim'][k]['accuracy_mean'] = np.mean(esn_results_d['reservoir_dim'][k]['accuracy'])
    esn_results_d['reservoir_dim'][k]['accuracy_std'] = np.std(esn_results_d['reservoir_dim'][k]['accuracy'])
    
    esn_results_d['reservoir_dim'][k]['time_mean'] = np.mean(esn_results_d['reservoir_dim'][k]['time'])
    esn_results_d['reservoir_dim'][k]['time_std'] = np.std(esn_results_d['reservoir_dim'][k]['time'])

In [19]:
for k, v in cbs_results_d['reservoir_dim'].items():
    cbs_results_d['reservoir_dim'][k]['accuracy_mean'] = np.mean(cbs_results_d['reservoir_dim'][k]['accuracy'])
    cbs_results_d['reservoir_dim'][k]['accuracy_std'] = np.std(cbs_results_d['reservoir_dim'][k]['accuracy'])
    
    cbs_results_d['reservoir_dim'][k]['time_mean'] = np.mean(cbs_results_d['reservoir_dim'][k]['time'])
    cbs_results_d['reservoir_dim'][k]['time_std'] = np.std(cbs_results_d['reservoir_dim'][k]['time'])

In [21]:
# save results

with open(os.path.join(RESULTS_PATH, RESULTS_FILE_1), 'wb') as fh:
    pickle.dump(esn_results_d, fh)
    
with open(os.path.join(RESULTS_PATH, RESULTS_FILE_2), 'wb') as fh:
    pickle.dump(cbs_results_d, fh)

In [22]:
# with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'rb') as fh:
#     esn_results_l = pickle.load(fh)

In [23]:
# The parameters do generalize well across different reservoir dims.
# Hence, they should be optimized for each reservoir dims.